In [83]:
import requests
import json
import pandas as pd
from openai import OpenAI
import math
import jsonlines
import random
import os


# 1. Getting OpenAI api to work

## 1.1 Info

In [19]:
URL = 'https://api.openai.com/v1/chat/completions'


## 1.2 Getting initial response

In [4]:
payload = {
    "model": "gpt-3.5-turbo",
    "messages": [
      {
        "role": "user",
        "content": "Act as Socrates and convince me, using the Socratic Method, about the nature of Justice. This has to be done using a question by question approach. Wait for me to answer you in order to give me a response guiding me towards your definition."
      }],
      "temperature" : 1.0,
      "n" : 1,
      "stream" : False,
      "presence_penalty" : 0,
      "frequency_penalty" : 0,
  }

headers = {
    "Content-Type" : "application/json",
    "Authorization" : f"Bearer {OPENAI_KEY}"
}


In [5]:
response = requests.post(URL, headers=headers, json=payload, stream=False)


In [6]:
bytes_obj = response.content


In [7]:
my_json = bytes_obj.decode('utf8')
print(my_json)


{
  "id": "chatcmpl-8PtNFWexzGtv0af2OKiFq4SdGlFmM",
  "object": "chat.completion",
  "created": 1701181749,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Socrates: Greetings, my friend. I am intrigued by your interest in justice. Let us embark on this philosophical journey together. \n\nFirstly, may I ask you, what is your current understanding of justice?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 58,
    "completion_tokens": 44,
    "total_tokens": 102
  }
}



In [8]:
data = json.loads(my_json)
message_data = data["choices"][0]["message"]
formatted_data = json.dumps(message_data, indent=4, sort_keys=True)
print(formatted_data)


{
    "content": "Socrates: Greetings, my friend. I am intrigued by your interest in justice. Let us embark on this philosophical journey together. \n\nFirstly, may I ask you, what is your current understanding of justice?",
    "role": "assistant"
}


In [31]:
content = message_data['content']


In [ ]:
# ''.join(list(message_data['content']))
# content_list = message_data["content"].split("\n\n")
# df = pd.DataFrame({"Speaker": ["Socrates" if i % 2 == 0 else "You" for i in range(len(content_list))],
#                    "Dialogue": content_list})
# df.set_index("Speaker", inplace=True)
# df['Dialogue'] = df['Dialogue'].str.replace(r'^(Socrates|You): ', '', regex=True)


In [32]:
dialogues = []
current_speaker = None
current_dialogue = ""

# Iterate through each line in the content
for line in content.split("\n"):
    # Check if the line starts with a known speaker
    if line.startswith(("Socrates:", "You:")):
        # If there is a current dialogue, add it to the list
        if current_dialogue:
            dialogues.append({"Speaker": current_speaker, "Dialogue": current_dialogue.strip()})

        # Update the current speaker and reset the current dialogue
        current_speaker, current_dialogue = line.split(":", 1)
    else:
        # If no speaker change, append the line to the current dialogue
        current_dialogue += " " + line

# Append the last dialogue to the list
if current_dialogue:
    dialogues.append({"Speaker": current_speaker, "Dialogue": current_dialogue.strip()})


In [40]:
dialogues
df_2 = pd.DataFrame(dialogues)


In [41]:
df_2


,Speaker,Dialogue
0,Socrates,"Greetings, my friend. I am intrigued by your i..."


## 1.3 Fine-tuning GPT

### 1.3.1 Load Data

In [59]:
Book_I_partI = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/Book_I_partI.csv')
Book_I_partI.name = 'Book_I_partI'
Book_I_partII = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_I_partII.csv')
Book_I_partII.name = 'Book_I_partII'
Book_I_partIII = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_I_partIII.csv')
Book_I_partIII.name = 'Book_I_partIII'
Book_I_partIV = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_I_partIV.csv')
Book_I_partIV.name = 'Book_I_partIV'
Book_I_partV = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_I_partV.csv')
Book_I_partV.name = 'Book_I_partV'
Book_I_partVI = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_I_partVI.csv')
Book_I_partVI.name = 'Book_I_partVI'
Book_II_partI = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_II_partI.csv')
Book_II_partI.name = 'Book_II_partI'
Book_II_partII = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_II_partII.csv')
Book_II_partII.name = 'Book_II_partII'
Book_II_partIII = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_II_partIII.csv')
Book_II_partIII.name = 'Book_II_partIII'
Book_II_partIV = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_II_partIV.csv')
Book_II_partIV.name = 'Book_II_partIV'
Book_III_partI = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_III_partI.csv')
Book_III_partI.name = 'Book_III_partI'
Book_III_partII = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_III_partII.csv')
Book_III_partII.name = 'Book_III_partII'
Book_IV_partI = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_IV_partI.csv')
Book_IV_partI.name = 'Book_IV_partI'
Book_IV_partII = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_IV_partII.csv')
Book_IV_partII.name = 'Book_IV_partII'
Book_V = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_V.csv')
Book_V.name = 'Book_V'
Book_VI_partI = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_VI_partI.csv')
Book_VI_partI.name = 'Book_VI_partI'
Book_VI_partII = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_VI_partII.csv')
Book_VI_partII.name = 'Book_VI_partII'
Book_VI_partIII = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_VI_partIII.csv')
Book_VI_partIII.name = 'Book_VI_partIII'
Book_VII = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_VII.csv')
Book_VII.name = 'Book_VII'
Book_VIII_partI = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_VIII_partI.csv')
Book_VIII_partI.name = 'Book_VIII_partI'
Book_VIII_partII = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_VIII_partII.csv')
Book_VIII_partII.name = 'Book_VIII_partII'
Book_IX_partI = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_IX_partI.csv')
Book_IX_partI.name = 'Book_IX_partI'
Book_IX_partII = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_IX_partII.csv')
Book_IX_partII.name = 'Book_IX_partII'
Book_X_partI = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_X_partI.csv')
Book_X_partI.name = 'Book_X_partI'
Book_X_partII = pd.read_csv('/root/code/LouisDeclee/EduCrate/data/raw_data_csv/BOOK_X_partII.csv')
Book_X_partII.name = 'Book_X_partII'


In [60]:
df_list = [
    Book_I_partI, Book_I_partII, Book_I_partIII, Book_I_partIV , Book_I_partV, Book_I_partVI,
    Book_II_partI, Book_II_partII, Book_II_partIII, Book_II_partIV,
    Book_III_partI, Book_III_partII,
    Book_IV_partI, Book_IV_partII,
    Book_V,
    Book_VI_partI, Book_VI_partII, Book_VI_partIII,
    Book_VII,
    Book_VIII_partI, Book_VIII_partII,
    Book_IX_partI, Book_IX_partII,
    Book_X_partI, Book_X_partII
]


In [61]:
df_names = [df.name for df in df_list]
df_names


['Book_I_partI',
 'Book_I_partII',
 'Book_I_partIII',
 'Book_I_partIV',
 'Book_I_partV',
 'Book_I_partVI',
 'Book_II_partI',
 'Book_II_partII',
 'Book_II_partIII',
 'Book_II_partIV',
 'Book_III_partI',
 'Book_III_partII',
 'Book_IV_partI',
 'Book_IV_partII',
 'Book_V',
 'Book_VI_partI',
 'Book_VI_partII',
 'Book_VI_partIII',
 'Book_VII',
 'Book_VIII_partI',
 'Book_VIII_partII',
 'Book_IX_partI',
 'Book_IX_partII',
 'Book_X_partI',
 'Book_X_partII']

### 1.3.2 Format Data

#### A) Droping NaN and Null

In [62]:
for df in df_list:
    df.dropna()


In [63]:
for df in df_list:
    null_check = df.isnull()
    any_null_rows = null_check.any(axis=1)
    rows_with_null = df[any_null_rows]
    null_counts = df.isnull().sum()
    any_null_value = df.isnull().any().any()
    print(df.name, any_null_value)


Book_I_partI False
Book_I_partII False
Book_I_partIII False
Book_I_partIV False
Book_I_partV False
Book_I_partVI False
Book_II_partI False
Book_II_partII False
Book_II_partIII False
Book_II_partIV False
Book_III_partI False
Book_III_partII False
Book_IV_partI False
Book_IV_partII False
Book_V False
Book_VI_partI False
Book_VI_partII False
Book_VI_partIII False
Book_VII False
Book_VIII_partI False
Book_VIII_partII False
Book_IX_partI False
Book_IX_partII False
Book_X_partI False
Book_X_partII True


In [66]:
for df in df_list:
    null_check = df.isnull()
    any_null_rows = null_check.any(axis=1)
    rows_with_null_indices = any_null_rows[any_null_rows].index
    print(df.name, rows_with_null_indices)


Book_I_partI Index([], dtype='int64')
Book_I_partII Index([], dtype='int64')
Book_I_partIII Index([], dtype='int64')
Book_I_partIV Index([], dtype='int64')
Book_I_partV Index([], dtype='int64')
Book_I_partVI Index([], dtype='int64')
Book_II_partI Index([], dtype='int64')
Book_II_partII Index([], dtype='int64')
Book_II_partIII Index([], dtype='int64')
Book_II_partIV Index([], dtype='int64')
Book_III_partI Index([], dtype='int64')
Book_III_partII Index([], dtype='int64')
Book_IV_partI Index([], dtype='int64')
Book_IV_partII Index([], dtype='int64')
Book_V Index([], dtype='int64')
Book_VI_partI Index([], dtype='int64')
Book_VI_partII Index([], dtype='int64')
Book_VI_partIII Index([], dtype='int64')
Book_VII Index([], dtype='int64')
Book_VIII_partI Index([], dtype='int64')
Book_VIII_partII Index([], dtype='int64')
Book_IX_partI Index([], dtype='int64')
Book_IX_partII Index([], dtype='int64')
Book_X_partI Index([], dtype='int64')
Book_X_partII Index([], dtype='int64')


In [65]:
good_col = ['Speaker', 'Text']

for df in df_list:
    cols_to_drop = [col for col in df.columns if col not in good_col]
    df.drop(columns=cols_to_drop, inplace=True)

    null_check = df.isnull()
    any_null_rows = null_check.any(axis=1)
    rows_with_null_indices = any_null_rows[any_null_rows].index
    df.drop(rows_with_null_indices, inplace=True)


#### B) Formatting for API

In [67]:
for i in range(len(df_list)):
    for index, row in df_list[i].iterrows():
        if row['Speaker'] == 'NARRATOR':
            df_list[i] = df_list[i].drop(index=index)
        else:
            pass


In [68]:
for df, df_name in zip(df_list, df_names):

    chat_messages = []
    system_message = "Act as Socrates and convince me, using the Socratic Method, about the nature of Justice. This has to be done using a question by question approach. Wait for me to answer you in order to give me a response guiding me towards your definition."

    chat_messages.append({"role": "system", "content": system_message})

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        # Define the role based on the speaker
        role = "assistant" if row['Speaker'] == "SOCRATES" else "user"

        # Create a message and append it to the list
        message = {"role": role, "content": row['Text']}
        chat_messages.append(message)

    # Convert the list of chat messages to JSON format
    chat_json = json.dumps(chat_messages)
    print(chat_json)
    chat = eval(chat_json)
    print(chat)
    final = {'messages' : chat}


[{"role": "system", "content": "Act as Socrates and convince me, using the Socratic Method, about the nature of Justice. This has to be done using a question by question approach. Wait for me to answer you in order to give me a response guiding me towards your definition."}, {"role": "assistant", "content": "I WENT down yesterday to the Piraeus with Glaucon the son of Ariston, that I might offer up my prayers to the goddess; and also because I wanted to see in what manner they would celebrate the festival, which was a new thing. I was delighted with the procession of the inhabitants; but that of the Thracians was equally, if not more, beautiful.  When we had finished our prayers and viewed the spectacle, we turned in the direction of the city; and at that instant Polemarchus the son of Cephalus chanced to catch sight of us from a distance as we were starting on our way home, and told his servant to run and bid us wait for him."}, {"role": "user", "content": "Polemarchus desires you to 

#### C) Split files into train and val jsonl files

In [73]:
def random_select(files, num_select_files):
    selected_indices = random.sample(range(len(files)), num_select_files)
    training_files = [files[i] for i in selected_indices]
    validation_files = [file for idx, file in enumerate(files) if idx not in selected_indices]
    return training_files, validation_files


In [77]:
document_name_json = []

for name in df_names:
    document_name_json.append(f'{name}.json')

num_select_files = int(.8*len(document_name_json))

training_files, validation_files = random_select(document_name_json, num_select_files)


In [86]:
[print("Training Files: " + file) for file in training_files]
[print("Validation Files: " + file) for file in validation_files]


Training Files: Book_II_partII.json
Training Files: Book_VII.json
Training Files: Book_IX_partI.json
Training Files: Book_I_partI.json
Training Files: Book_I_partVI.json
Training Files: Book_II_partI.json
Training Files: Book_VIII_partI.json
Training Files: Book_III_partI.json
Training Files: Book_X_partI.json
Training Files: Book_X_partII.json
Training Files: Book_VI_partI.json
Training Files: Book_III_partII.json
Training Files: Book_I_partV.json
Training Files: Book_VI_partIII.json
Training Files: Book_V.json
Training Files: Book_II_partIV.json
Training Files: Book_I_partII.json
Training Files: Book_IV_partII.json
Training Files: Book_I_partIV.json
Training Files: Book_VI_partII.json
Validation Files: Book_I_partIII.json
Validation Files: Book_II_partIII.json
Validation Files: Book_IV_partI.json
Validation Files: Book_VIII_partII.json
Validation Files: Book_IX_partII.json


[None, None, None, None, None]

In [89]:
directory = '/root/code/LouisDeclee/EduCrate/data/raw_data_json'

#Initiate a new blank list for storing json data in list format
train_json_list=[]
val_json_list=[]

for dirpath, subdirs, files in os.walk(directory):
    print(dirpath)
    for file in training_files:
        if file.endswith(".json"):
            with open(os.path.join(dirpath, file)) as train_json_file:
                train_data = json.load(train_json_file)
                train_json_list.append(train_data)
    for file in validation_files:
        if file.endswith(".json"):
            with open(os.path.join(dirpath, file)) as val_json_file:
                val_data = json.load(val_json_file)
                val_json_list.append(val_data)

#Now, output the list of json data into a single jsonl file
with open('/root/code/LouisDeclee/EduCrate/data/raw_data_jsonl/train.jsonl', 'w') as train_outfile:
    for entry in train_json_list:
        json.dump(entry, train_outfile)
        train_outfile.write('\n')
with open('/root/code/LouisDeclee/EduCrate/data/raw_data_jsonl/val.jsonl', 'w') as val_outfile:
    for entry in val_json_list:
        json.dump(entry, val_outfile)
        val_outfile.write('\n')


/root/code/LouisDeclee/EduCrate/data/raw_data_json


### 1.3.3 API call

In [91]:
client = OpenAI(api_key=OPENAI_KEY)

train_file = 'train.jsonl'
val_file = 'val.jsonl'
files = [train_file, val_file]

ids = []

for file in files:
    object = client.files.create(
    file=open(f'/root/code/LouisDeclee/EduCrate/data/raw_data_jsonl/{file}', "rb"),
    purpose="fine-tune"
    )
    ids.append(object.id)


# client.fine_tuning.jobs.create(
# training_file=id,
# model="gpt-3.5-turbo"
# )


In [92]:
existing_jobs = client.fine_tuning.jobs.list()

for job in existing_jobs:
    print(f"Job ID: {job.id}, Status: {job.status}")


Job ID: ftjob-JbYxq6dtW3UUKJYve92YI212, Status: failed
Job ID: ftjob-yKWXbuzpL1nIHNX3LhQaPyam, Status: failed
Job ID: ftjob-PN2Vg7dGTYMfY93SjkxH0dbC, Status: failed
Job ID: ftjob-uMHLcrGpeWOiMZ50DMyLuGUA, Status: failed


# Getting insights into Constitutional AIs

# Testing Finetuning

In [75]:
df = pd.read_csv('/root/code/LouisDeclee/EduCrate/raw_data_csv/BOOK_I_partVI.csv')
df.head(3)

chat_messages = []
system_message = "Act as Socrates and convince me, using the Socratic Method, about the nature of Justice. This has to be done using a question by question approach. Wait for me to answer you in order to give me a response guiding me towards your definition."

chat_messages.append({"role": "system", "content": system_message})

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Define the role based on the speaker
    role = "assistant" if row['Speaker'] == "SOCRATES" else "user"

    # Create a message and append it to the list
    message = {"role": role, "content": row['Text']}
    chat_messages.append(message)

# Convert the list of chat messages to JSON format
chat_json = json.dumps(chat_messages)

chat = eval(chat_json)
final = {'messages' : chat}

with open('/root/code/LouisDeclee/EduCrate/test_data/final.json', 'w') as file:
    file.write(json.dumps(final))

client = OpenAI(api_key=OPENAI_KEY)

client.files.create(
  file=open('/root/code/LouisDeclee/EduCrate/test_data/final.json', "rb"),
  purpose="fine-tune"
)

client.fine_tuning.jobs.create(
    training_file='file-exzaCyT8hljCcKr2WJi8uYNC',
    model="gpt-3.5-turbo"
)


# Code that is ont needed but could be usefull in case

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [ ]:
# document =
# splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# chunks = splitter.split_documents(document)


In [133]:
# Book_I_partI.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_I_partI.json')
# Book_I_partII.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_I_partII.json')
# Book_I_partIII.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_I_partIII.json')
# Book_I_partIV.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_I_partIV.json')
# Book_I_partV.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_I_partV.json')
# Book_I_partVI.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_I_partVI.json')
# Book_II_partI.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_II_partI.json')
# Book_II_partII.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_II_partII.json')
# Book_II_partIII.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_II_partIII.json')
# Book_II_partIV.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_II_partIV.json')
# Book_III_partI.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_III_partI.json')
# Book_III_partII.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_III_partII.json')
# Book_IV_partI.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_IV_partI.json')
# Book_IV_partII.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_IV_partII.json')
# Book_V.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_V.json')
# Book_VI_partI.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_VI_partI.json')
# Book_VI_partII.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_VI_partII.json')
# Book_VI_partIII.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_VI_partIII.json')
# Book_VII.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_VII.json')
# Book_VIII_partI.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_VIII_partI.json')
# Book_VIII_partII.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_VIII_partII.json')
# Book_IX_partI.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_IX_partI.json')
# Book_IX_partII.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_IX_partII.json')
# Book_X_partI.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_X_partI.json')
# Book_X_partII.to_json(r'/root/code/LouisDeclee/EduCrate/raw_data_json/Book_X_partII.json')


In [32]:
# List of your JSON files
data_files = [
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_I_partI.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_I_partII.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_I_partIII.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_I_partIV.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_I_partV.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_I_partVI.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_II_partI.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_II_partII.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_II_partIII.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_II_partIV.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_III_partI.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_III_partII.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_IV_partI.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_IV_partII.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_V.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_VI_partI.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_VI_partII.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_VI_partIII.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_VII.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_VIII_partI.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_VIII_partII.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_IX_partI.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_IX_partII.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_X_partI.json",
    "/root/code/LouisDeclee/EduCrate/raw_data_json/Book_X_partII.json"
]


In [ ]:
# with open(file_name, 'w') as file:
#     file.write(json.dumps(final))
